In [3]:
# !pip install git+https://github.com/tflearn/tflearn.git

  Cloning https://github.com/tflearn/tflearn.git to /tmp/pip-WWzo2s-build
  Running setup.py install for tflearn ... - done
You are using pip version 8.1.1, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import numpy as np
import tflearn

### Variables

In [5]:
data = np.load('/local_data/alexnet-feats/iaprtc_alexfc7.npz')
D = open('/local_data/alexnet-feats/iaprtc_dictionary.txt').read().splitlines()
train_ims = [ im.split('/')[-1] for im in open('/local_data/alexnet-feats/iaprtc_trainlist.txt').read().splitlines() ]
test_ims = [ im.split('/')[-1] for im in open('/local_data/alexnet-feats/iaprtc_testlist.txt').read().splitlines() ]
xTr = data['xTr'].T
yTr = data['yTr'].T
xTe = data['xTe'].T
yTe = data['yTe'].T
wc = yTr.sum(axis=0)+0.00

# Read w2vec
output_dict = {}
for i, line in enumerate(open('/work/attalos/models/glove.42B.300d.txt')):
    first_word = line[:line.find(' ')]
    if first_word in D:
        line = line.strip().split(' ')
        output_dict[line[0]] = np.array([ float(j) for j in line[1:]])
    

imfeatsize = xTr.shape[1]
vocabsize = yTr.shape[1]
wordvecsize = output_dict[output_dict.keys()[0]].shape[0]

IOError: [Errno 2] No such file or directory: '/local_data/alexnet-feats/iaprtc_alexfc7.npz'

In [4]:
BATCH_SIZE = 256
HIDDEN_SIZE = 300
RATE = .1

### For a single feature, get the related training vectors

Usage:  (im, veclist) = get_training_vectors( index_number )

### For multiple features (in the list of indices), for each image feature, sum the training vectors

In [5]:
# Given an index into the corpus, get the corresponding image feature and all word vectors from it
def get_training_vectors(index):
    yTrList = []
    for i in np.nonzero( yTr[index] )[0]:
        yTrList.append( output_dict[ D[i] ] )
    return (xTr[index], np.array(yTrList))

def get_training_batch(indices):
    
    feats = []#np.zeros((len(indices),imfeatsize))
    labels = []#np.zeros((len(indices),wordvecsize))
    for j, i in enumerate(indices):
        #feats[j, :], wordfeats = get_training_vectors(i)
        feat, wordfeats = get_training_vectors(i)
        feats.append(feat)
        #labels[j, :] = wordfeats.sum(axis=0)
        labels.append(wordfeats.sum(axis=0))
    return (feats, labels)

In [6]:
# Given an index into the corpus, get the corresponding image feature and all word vectors from it
def get_test_vectors(index):
    yTeList = []
    for i in np.nonzero( yTe[index] )[0]:
        yTeList.append( output_dict[ D[i] ] )
    return (xTe[index], np.array(yTeList))

def get_test_batch(indices):
    
    feats = []#np.zeros((len(indices),imfeatsize))
    labels = []#np.zeros((len(indices),wordvecsize))
    for j, i in enumerate(indices):
        #feats[j, :], wordfeats = get_training_vectors(i)
        feat, wordfeats = get_test_vectors(i)
        feats.append(feat)
        #labels[j, :] = wordfeats.sum(axis=0)
        labels.append(wordfeats.sum(axis=0))
    return (feats, labels)

In [7]:
print imfeatsize

4096


In [ ]:
# Linear Regression graph
net = tflearn.input_data(shape=[None, imfeatsize])
net = tflearn.layers.core.fully_connected(net, 300, activation='sigmoid')
net = tflearn.layers.core.fully_connected(net, 300, activation='sigmoid')
#net = tflearn.layers.core.fully_connected(net, 300)
net = tflearn.regression(net, optimizer='adam', loss='binary_crossentropy')

model = tflearn.DNN(net, tensorboard_verbose=3, tensorboard_dir='/tmp/tensorboard/')
# loss_over_time = []

# # print("\nTest prediction for y = 3.2 and y = 4.5:")


    # print(m.predict([3.2, 4.5]))

In [ ]:
imbatch, wordbatch = get_training_batch(np.arange(0, yTr.shape[0]))
imbatch_te, wordbatch_te = get_test_batch(np.arange(0, yTe.shape[0]))
model.fit(imbatch, wordbatch, validation_set=(imbatch_te, wordbatch_te), batch_size=128)

Training Step: 39  | total loss: 0.65213
| Adam | epoch: 000 | loss: 0.65213 -- iter: 04992/17665


In [ ]:
# batch_size = 10000
# for i in range(5):
#     print(i)
#     # Get a batch for training
#     imbatch, wordbatch = get_training_batch(np.random.randint(0, yTr.shape[0], batch_size))
#     #print(len(imbatch), len(imbatch[0]))
#     model.fit(imbatch, wordbatch)
#     #print("\nRegression result:")
#     #print("Y = " + str(model.get_weights(linear.W)) +
#     #  ".X + " + str(model.get_weights(linear.b)))